<a href="https://colab.research.google.com/github/Rabbia-Ijaz/Timetable-Generator-Genetic-Algorithm/blob/main/TimetableGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import zipfile
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pickle5
import pickle5 as pickle
import pandas as pd
import numpy as np
import random
from itertools import groupby
from operator import itemgetter
from pandas import ExcelWriter
from openpyxl import load_workbook

In [ ]:
#Initial Data 
Rooms=[]
Slots=["08:30","09:30","10:30","11:30","12:30","01:30","02:30","03:30","04:30","05:30"]
Days=["Monday","Tuesday","Wednesday","Thursday","Friday"]
Sections=[]
Courses=[]
Teachers=[]
UniqueClasses=[]
def unpickling_data(): #inputFile
    file = open("/content/drive/My Drive/Colab Notebooks/input-data-all.pkl",'rb')
    new_data = pickle.load(file)
    file.close()
    return new_data

pd.set_option("display.max_rows", None, "display.max_columns", None)


R=np.array(unpickling_data())
orgDF = pd.DataFrame(data=R,index=None, columns=["Professor","Course","Section","Duration","Classrooms"])
df=orgDF.copy()
df.insert(0, "RoomAlloted",value=False) 
print(df)

df=df.drop([46, 82,83,71,96,97,98,19,114,102,103])
df=df.reset_index(drop=True)
total=109

#display(df)

for i in range(0,total): #Rooms
  for j in range(0, len(df['Classrooms'][i])):
    Rooms.append(df['Classrooms'][i][j])
Rooms=np.array(Rooms)
Rooms=np.unique(Rooms)
#print(len(Rooms))

for i in range(0,total): #Courses
  Courses.append(df['Course'][i])
Courses=np.array(Courses)
Courses=np.unique(Courses)
#print(Courses)

for i in range(0,total): #Teachers
  Teachers.append(df['Professor'][i])
Teachers=np.array(Teachers)
Teachers=np.unique(Teachers)
#print(Teachers)

for i in range(0,total): #Sections
  Sections.append(df['Section'][i])
Sections=np.array(Sections)
Sections=np.unique(Sections)
#print(Sections)

for i in range(0,total): #Courses+Sections
  UniqueClasses.append(df['Professor'][i]+" "+df['Course'][i]+'-('+df['Section'][i]+')')
UniqueClasses=np.array(UniqueClasses)
UniqueClasses=np.unique(UniqueClasses)
#print(UniqueClasses)
#print(len(UniqueClasses))



     RoomAlloted                   Professor    Course Section Duration  \
0          False    Mr. Usman Zafar Chaudhry  BIOP-595       A        1   
1          False    Mr. Usman Zafar Chaudhry  BIOP-595       B        1   
2          False    Mr. Usman Zafar Chaudhry  BIOP-595       C        1   
3          False           Mr. Fahad Shahzad  BIOP-595       D        1   
4          False      Dr. Zohaib Zafar Iqbal  BIOP-595       E        1   
5          False  Mr. Hafiz M. Tayyeb Nadeem  BIOP-595       F        1   
6          False          Mr. Shoaib Mehboob  SOCW-199       A        1   
7          False       Mr. Mehmood Ur Rehman  SOCW-199       B        1   
8          False       Mr. Mehmood Ur Rehman  SOCW-199       C        1   
9          False       Mr. Mehmood Ur Rehman  SOCW-199       D        1   
10         False          Dr. Muhammad Aleem  SOCW-199       E        1   
11         False          Dr. Muhammad Aleem  SOCW-199       F        1   
12         False         

In [ ]:
def Con1(Gen,Instructor): #No teacher can hold two classes at the same time

  ind=Gen[Gen['Professor']==Instructor].index.values
  count=len(ind)
  k=0
  SlotInd=[]
  while k<count:
    if pd.notna(Gen.loc[ind[k],'Slot']):
      time=Gen.loc[ind[k],'Slot']
      SlotInd.append(Slots.index(time))
    k=k+1    
  S=sorted(SlotInd)
  #print(S)
  size = len(S) 
      
    # looping till length - 2 
  if size >=2:
    for i in range(size - 1): 
          if S[i] == S[i + 1]:
            return 0
  return 10 

def Con2(Slot): #No section can listen for two classes at the same time
  if pd.isnull(Slot)==True:
    return 10
  else:
    return 0

def Con3(Slot): #No classroom can receive two classes at the same time
  if pd.isnull(Slot)==True:
    return 10
  else:
    return 0

def Con4(Gen,Instructor): #No teacher can hold three consecutive classes
  ind=Gen[Gen['Professor']==Instructor].index.values
  count=len(ind)
  k=0
  SlotInd=[]
  exist=True
  while k<count:
    if pd.isnull(Gen.loc[ind[k],'Slot']):
      exist=False
      break
    time=Gen.loc[ind[k],'Slot']
    SlotInd.append(Slots.index(time))
    k=k+1
  if exist==True:
    S=sorted(SlotInd)
    for k, g in groupby(enumerate(S), lambda ix : ix[0] - ix[1]): #find list of consective slots
      L=list(map(itemgetter(1), g))
      
      if len(L)==3:
        return 0
  return 10

def Con5(): #There will be no class before 8:30 am and after 5:30 pm.
  if Slots[0]=="08:30" and Slots[len(Slots)-1]=="05:30":
    return 10
  else:
    return 0

def Con6(): #University will remain close as there will be no class on weekends (Sat, Sun)
  if Days[0]=="Monday" and Days[len(Days)-1]=="Friday":
    return 10
  else:
    return 0

def SCon1(Gen,Instructor): #No Section can hold three consecutive classes
  if Con4(Gen,Instructor) ==10:
    return 5
  else:
    return 0

In [ ]:
def FirstGeneration(Timetable): #Random Population
  Gen=pd.DataFrame(data=None,index=None, columns=["Professor","Course","Section","Slot","Room","Classrooms","Fitness"])
  Gen.loc[:,'Professor']=Tdf2.loc[:,'Professor']
  Gen.loc[:,'Course']=Tdf2.loc[:,'Course']
  Gen.loc[:,'Section']=Tdf2.loc[:,'Section']
  Gen.loc[:,'Classrooms']=Tdf2.loc[:,'Classrooms']
  for i in range(0,total):
    j=random.randint(0,len(Tdf2.loc[i,'Classrooms'])-1)
    Room=Tdf2.loc[i,'Classrooms'][j]
    k=random.randint(0,len(Slots)-1)
    while pd.notna(Timetable.loc[Room,Slots[k]]) or Tdf2.loc[i,'RoomAlloted']==True:
      j=random.randint(0,len(Tdf2.loc[i,'Classrooms'])-1)
      Room=Tdf2.loc[i,'Classrooms'][j]
      k=random.randint(0,len(Slots)-1)
    if pd.isnull(Timetable.loc[Room,Slots[k]]) and Tdf2.loc[i,'RoomAlloted']==False:
      Fitness=0                                                         #initialFitness
      Fitness=Fitness+Con2(Timetable.loc[Room,Slots[k]])                #Constraint 2
      Fitness=Fitness+Con3(Timetable.loc[Room,Slots[k]])                #Constraint 3 
      Tdf2.loc[i,'RoomAlloted']=True
      Timetable.loc[Room,Slots[k]]= Tdf2.loc[i,'Professor']
      Gen.loc[i,'Slot']=Slots[k]
      Gen.loc[i,'Room']=Room   
      Fitness=Fitness+Con1(Gen,Tdf2.loc[i,'Professor'])                 #Constraint 1
      Fitness=Fitness+Con4(Gen,Tdf2.loc[i,'Professor'])                 #Constraint 4
      Fitness=Fitness+Con5()                                            #Constraint 5
      Fitness=Fitness+Con6()                                            #Constraint 6

      Fitness=Fitness+SCon1(Gen,Tdf2.loc[i,'Professor'])                #Soft Constraint 1
      Gen.loc[i,'Fitness']=Fitness
  return Gen

def Randomness(Timetable,Gen,P1,EClass): 
  Gen.loc[P1,'Slot']=Gen.loc[EClass,'Slot']
  Gen.loc[P1,'Selected']='1'
  Gen.loc[EClass,'Selected']='2'

  Fitness=0
  Fitness=Fitness+Con2(Timetable.loc[Gen.loc[P1,'Room'],Gen.loc[P1,'Slot']])                #Constraint 2
  Fitness=Fitness+Con3(Timetable.loc[Gen.loc[P1,'Room'],Gen.loc[P1,'Slot']])                #Constraint 3 
  Fitness=Fitness+Con1(Gen,Gen.loc[P1,'Professor'])                                         #Constraint 1
  Fitness=Fitness+Con4(Gen,Gen.loc[P1,'Professor'])                                         #Constraint 4
  Fitness=Fitness+Con5()                                                                    #Constraint 5
  Fitness=Fitness+Con6()                                                                    #Constraint 6

  Fitness=Fitness+SCon1(Gen,Gen.loc[P1,'Professor'])                                        #Soft Constraint 1
  Gen.loc[P1,'Fitness']=Fitness

def Crossover(Gen,Timetable):
  Gen=Gen.sort_values(by=['Fitness'])
  Gen=Gen.reset_index(drop=True)
  elite=Gen[Gen['Fitness']==65].index.values
  j=elite[0]
  while j<=elite[-1]:
    Timetable.loc[Gen.loc[j,'Room'],Gen.loc[j,'Slot']]=Gen.loc[j,'Professor']
    j=j+1
  i=0
  while i<elite[0]:
    P1=random.randint(0,elite[0]-1)
    EClass=random.randint(elite[0],elite[-1])
    while Gen.loc[P1,'Selected']=='1'  or Gen.loc[EClass,'Selected']=='2':
      P1=random.randint(0,elite[0]-1)
      EClass=random.randint(elite[0],elite[-1])
    if Gen.loc[P1,'Selected']=='0'  and Gen.loc[EClass,'Selected']=='0':
      Randomness(Timetable,Gen,P1,EClass)

    i=i+1
  return Gen
def GenTimetable(Gen):
  T=pd.DataFrame(data=None,index=Rooms,columns=Slots)
  for i in range(total):
    T.loc[Gen.loc[i,'Room'],Gen.loc[i,'Slot']]=Gen.loc[i,'Professor']

  return T

#Main
Week=[]
for i in range(0,5):
  print("******** "+Days[i]+" ********")
  Tdf2=df.copy(deep=True)
  TimetableDF2=pd.DataFrame(data=None,index=Rooms,columns=Slots)

  FirstGen=FirstGeneration(TimetableDF2)
  TCount=1
  while len(FirstGen["Fitness"].unique())>1 and TCount<5000:  #
    FirstGen.insert(0,'Selected','0',allow_duplicates=True)
    FirstGen.insert(1,'RoomAlloted',value=False)
    Timetable2=pd.DataFrame(data=None,index=Rooms,columns=Slots)
    FirstGen = Crossover(FirstGen,Timetable2)
    FirstGen = FirstGen.drop('Selected', 1)
    FirstGen = FirstGen.drop('RoomAlloted', 1)
    print(TCount)
    TCount=TCount+1

  TimetableFinal=GenTimetable(FirstGen)
  Week.append(TimetableFinal)
  import pandas as pd
  import numpy as np

  path = r"/content/drive/My Drive/Colab Notebooks/A03-17I-0293.xlsx"

  book = load_workbook(path)
  writer = pd.ExcelWriter(path, engine = 'openpyxl')
  writer.book = book
  Week[i].to_excel(writer, sheet_name = Days[i])
  writer.save()
writer.close()

#display(TimetableFinal)